In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor 

/Users/ningsam/anaconda3/envs/YichenWang/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/3w/bkk9z93s1294lzphg00_yxnw0000gn/T/ipykernel_96929/4243316776.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
target = pd.read_csv("target.csv")
ds = pd.read_csv("/Users/ningsam/Desktop/Winter_2024/PIC16B/df_2023_h1_v2.csv")

In [3]:
target.describe()

,Adj Close
count,250.000000
mean,423.693802
std,24.340258
min,373.700256
25%,404.687393
50%,425.332275
75%,442.607903
max,476.690002


In [4]:
# for i in ds["Date"].unique():
#     print(i)

In [5]:
ds.columns

Index(['[QUOTE_UNIXTIME]', '[EXPIRE_UNIX]', '[EXPIRE_DATE]', '[STRIKE]',
       '[UNDERLYING_LAST]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]', '[C_THETA]',
       '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_BID]', '[C_ASK]', '[P_DELTA]',
       '[P_GAMMA]', '[P_VEGA]', '[P_THETA]', '[P_RHO]', '[P_IV]', '[P_VOLUME]',
       '[P_BID]', '[P_ASK]'],
      dtype='object')

In [6]:
# Rename ds's "[EXPIRE_DATE]" to "Date" 
# NOTE: Here the date refers to the expiration date of the option
ds.rename(columns={"[EXPIRE_DATE]": "Date"}, inplace=True)

In [7]:
# Sort both df per Date col
ds = ds.sort_values(by="Date").reset_index(drop=True)
target = target.sort_values(by="Date").reset_index(drop=True)
# Remove the Unnamed column
ds = ds.loc[:, ~ds.columns.str.contains('^Unnamed')]  
ds['Date'] = pd.to_datetime(ds['Date'])
target['Date'] = pd.to_datetime(target['Date'])

In [8]:
ds_new = pd.merge(ds, target, on='Date', how='left')

In [9]:
ds_new

,[QUOTE_UNIXTIME],[EXPIRE_UNIX],Date,[STRIKE],[UNDERLYING_LAST],[C_DELTA],[C_GAMMA],[C_VEGA],[C_THETA],[C_RHO],...,[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[P_BID],[P_ASK],Adj Close
0,-1.691600,-1.531564,2023-01-03,-1.054517,-2.406592,1.054125,-0.052714,0.000146,-0.391189,-0.307878,...,1.120791,-0.013192,0.001135,0.086874,0.592295,3.380989,-0.100406,-0.563368,-0.566211,375.118713
1,-1.691600,-1.531564,2023-01-03,0.141988,-2.406592,-1.302375,-0.388302,-0.000293,0.804031,-0.308329,...,-1.331602,-0.013434,0.001098,0.090668,0.592878,-1.037447,-0.099559,-0.005324,0.011661,375.118713
2,-1.691600,-1.531564,2023-01-03,0.130141,-2.406592,-1.302766,-0.385148,-0.000318,0.809161,-0.308903,...,-1.331602,-0.013434,0.001098,0.090668,0.592878,-1.037447,-0.099728,-0.022073,-0.007267,375.118713
3,-1.691600,-1.531564,2023-01-03,0.118294,-2.406592,-1.304380,-0.387672,-0.000283,0.800571,-0.308799,...,-1.331602,-0.013434,0.001098,0.090668,0.592878,-1.037447,-0.090071,-0.043390,-0.026195,375.118713
4,-1.691600,-1.531564,2023-01-03,0.106448,-2.406592,-1.303866,-0.385148,-0.000300,0.809758,-0.308329,...,-1.331602,-0.013434,0.001098,0.090668,0.592878,-1.037447,-0.098712,-0.060139,-0.045123,375.118713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331604,-0.470746,2.332822,2023-12-29,-1.350682,-0.792335,0.908009,-0.323330,0.028591,0.327876,1.567932,...,0.940700,-0.011274,0.027561,0.073490,-0.464073,0.053898,-0.100406,-0.487046,-0.488228,475.309998
331605,-0.470746,2.332822,2023-12-29,-1.291449,-0.792335,0.893192,-0.316391,0.030232,0.305088,1.583821,...,0.922980,-0.011194,0.029314,0.072979,-0.552965,0.038197,-0.099728,-0.480575,-0.481792,475.309998
331606,-0.470746,2.332822,2023-12-29,-1.232216,-0.792335,0.878326,-0.310083,0.031854,0.294231,1.598922,...,0.905580,-0.010968,0.031117,0.071872,-0.645241,0.018808,-0.100067,-0.473152,-0.475167,475.309998
331607,-0.470746,2.332822,2023-12-29,-1.113750,-0.792335,0.843019,-0.294943,0.035285,0.256172,1.624471,...,0.868226,-0.010533,0.034955,0.070434,-0.859345,-0.016721,-0.100236,-0.462113,-0.453968,475.309998


In [10]:
# Sequence creation
def seq(data, seq_length):
    xseq = []
    yseq = []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xseq.append(x)
        yseq.append(y)
    return np.array(xseq), np.array(yseq)

In [11]:
features = ds_new[['[QUOTE_UNIXTIME]', '[EXPIRE_UNIX]', '[STRIKE]', '[UNDERLYING_LAST]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]','[C_BID]', '[C_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[P_BID]', '[P_ASK]']].values
target=ds_new["Adj Close"]
seq_length = 10 # Number of days to look back
X,y = seq(features, seq_length)
y = y[:,np.newaxis] # This is to reshape y to be a 2D array

In [12]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Then, split the training + validation into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 is roughly 15% of the original dataset


In [13]:
X_train = X_train.astype(float)  # Convert to float if it's a numpy array
X_train = torch.tensor(X_train, dtype=torch.float)  # Then convert to a PyTorch tensor
y_train = torch.tensor(y_train, dtype=torch.float)
X_val = torch.tensor(X_val, dtype=torch.float)
y_val = torch.tensor(y_val, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

torch.Size([232110, 10, 22]) torch.Size([232110, 1, 22]) torch.Size([49748, 10, 22]) torch.Size([49748, 1, 22]) torch.Size([49740, 10, 22]) torch.Size([49740, 1, 22])


In [14]:
y_train = y_train.squeeze(1)  # Removes the unnecessary middle dimension
y_val = y_val.squeeze(1)
y_test = y_test.squeeze(1)

In [15]:
print("Train shapes:", X_train.shape, y_train.shape)
print("Validation shapes:", X_val.shape, y_val.shape)
print("Test shapes:", X_test.shape, y_test.shape)

Train shapes: torch.Size([232110, 10, 22]) torch.Size([232110, 22])
Validation shapes: torch.Size([49748, 10, 22]) torch.Size([49748, 22])
Test shapes: torch.Size([49740, 10, 22]) torch.Size([49740, 22])


In [16]:
class RNN_2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=22,dropout=0.5):
        super(RNN_2, self).__init__()
        # hidden layer size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,dropout=dropout)
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        test_h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        print(f"test_h0 has shape,{test_h0.shape}")  # Should work without error and print the shape)
        # initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        output, _ = self.lstm(x, (h0.detach(), c0.detach()))
        output = self.fc(output[:, -1, :])
        return output
    
# Model instantiation
model = RNN_2(input_size=X_train.shape[2], hidden_size=50, num_layers=3)


In [17]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# For early stopping
patience = 20
optimal_val_loss = np.inf
current_patience = 0
# Training phase
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    # print(outputs.shape)
    # print(y_train.shape)

    loss = criterion(outputs, y_train)
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)
        # Early stopping
        if val_loss < optimal_val_loss:
            optimal_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience == patience:
                print(f'Early stopping at epoch {epoch+1}')
                # load best model
                model.load_state_dict(torch.load('best_model.pt'))
                break
    
    
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Val Loss: {val_loss.item()}')


test_h0 has shape,torch.Size([3, 232110, 50])


test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,torch.Size([3, 49748, 50])
Epoch [10/100], Loss: 0.990469753742218, Val Loss: 0.8920930624008179
test_h0 has shape,torch.Size([3, 232110, 50])
test_h0 has shape,to

In [18]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)
    print(f'Test Loss: {test_loss.item()}')
criterion = torch.nn.MSELoss()
mse_loss = criterion(predictions, y_test)
print(f"MSE Loss: {mse_loss.item()}")
# R2 score
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print(f"R2 Score: {r2}")

test_h0 has shape,torch.Size([3, 49740, 50])
Test Loss: 0.6241307854652405
MSE Loss: 0.6241307854652405
R2 Score: 0.441430819532
